In [2]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium
# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,014 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,803 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
G

In [4]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
from tqdm import tqdm
import logging
import traceback
from selenium.common.exceptions import NoSuchElementException

In [5]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Files-Required/client project/problematics')

Mounted at /content/drive


In [6]:
unmerged = pd.read_csv('/content/drive/MyDrive/Files-Required/client project/problematics/unmerged.csv')

In [9]:
unmerge = unmerged['Search string for Domain'].to_list()

In [10]:
len(unmerge)

4896

In [49]:
unmerge[4895] # max index

'trial-harbour-tas-7469'

In [11]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

In [ ]:
def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [35]:
unmerge[2]

'cullen-bay-nt-0820'

In [ ]:
driver = driver_setup()
driver.set_page_load_timeout(60)

driver.get('https://www.google.com')

links = []

In [ ]:
pbar = tqdm(total=len(unmerge), desc='Searching', unit='query')

for i in range(len(unmerge)):

  driver.get('https://www.google.com')

  search_input = driver.find_element(By.NAME, 'q')
  search_input.send_keys(unmerge[i]+' suburb profile & local reviews site:homely.com.au')
  search_input.submit()

  # Wait for search results to load
  time.sleep(2)
  pbar.update(1)
  # Find the specific search result by the common string
  desired_result = None
  for result in driver.find_elements(By.CSS_SELECTOR, 'div.g'):
      url = result.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
      if 'homely.com.au' in url:
          desired_result = url
          break
  if desired_result is not None:

    file_name = 'googlelinks1.csv'
    tempdf = pd.DataFrame({'Link': desired_result},index=[0])
    save_data_to_disk(tempdf, file_name)
    tempdf = pd.DataFrame()
    driver.delete_all_cookies()